## Importation des librairies

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import datetime
from textblob import TextBlob
import pandas as pd
import json
import numpy as np
import re

## Scraper générique

Le but de cette fonction est de récupérer plusieurs informations sur tous les articles disponibles sur internet quelque soit le site. <br>
Pour chaque article, la fonction essaie de récupérer:
    - "art_content": le contenu de l'article
    - "art_content_html": le contenu de l'article avec les balises html
    - "art_published_datetime": la date de publication ou de modification de l'article et sinon le jour d'utilisation du scraper
    - "art_lang": la langue de l'article
    - "art_title": le titre de l'article
    - "art_url": l'url de l'article
    - "src_name": le nom du site internet sur lequel est l'article
    - "src_type": la façon de récupérer l'article (xpath_source pour tous les articles)
    - "src_url": l'url de base du site internet
    - "art_img": l'image associée à l'article
    - "art_auth": le ou les auteurs de l'article
    - "art_tag": les tags associés à l'article

Pour chaque information la fonction teste si des balises sont présentes dans le code source de l'article. Ces balises sont celles qui contiennent l'information voulue dans la plupart des codes des articles que l'on a rencontré pendant le projet. Enfin si aucune des balises recherchées ne sont disponibles dans le code source, la fonction renvoie np.nan (Not A Number) qui est équivalent à un None.
Pour ce qui est du contenu, on a essayé différentes façons de le récupérer, elles seront expliqués plus loin dans le notebook.

### Fonction complète du scraper générique

In [ ]:
def scrap_generic(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Getting content
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li']
    if soup.find('article') is not None:
        content_html = soup.find('article')
        if soup.find('article').find_all(list_balises, recursive = False):
            content = soup.find('article').find_all(list_balises, string = True)
            content = ' '.join(tag.text for tag in content)
        else:
            list_p = soup.find('article').find_all('div', recursive = 'False')
            print([p.parent.name for p in list_p])
            list_parent_p = [p.parent for p in list_p]
            print(list_parent_p)
            if len(set(list_parent_p)) == 1:
                content_html = list_parent_p[0]
                print(content_html)
                content = ' '.join(
                    tag.text for tag in content_html.children if tag.name in list_balises)
            else:
                content_html = soup.find('article')
                content = [el.get_text() for el in list_parent_p]
                content = ' '.join(content)
    else:
        list_div = list()
        for el in soup.find_all('div'):
            if el.find_all('p', recursive = False):
                list_div.append(el)
        index_max = np.argmax([len(block.find_all('p')) for block in list_div])
        content_html = list_div[index_max]
        content = ' '.join(
            tag.text for tag in content_html.children if tag.name in list_balises)
    content = content.replace('\xa0', '').replace('\t', '').replace('\r', '').strip()
    content_html_str = str(content_html)
    
    # Getting date of publication
    if soup.find("meta", {"property"
                          "article:modified_time"}) is not None:
        date = soup.find("meta", {"property"
                                  "article:modified_time"})["content"]
    elif soup.find("meta", {"property": "article:published_time"}) is not None:
        date = soup.find("meta", {"property": "article:published_time"})[
            "content"]
    else:
        date = datetime.date.today()

    # Getting language
    art_lang = TextBlob(content).detect_language()

    # Getting title
    if soup.find("meta", {"property": "og:title"}) is not None:
        title = soup.find("meta", {"property": "og:title"})["content"]
    elif soup.find("title") is not None:
        title = soup.find("title").get_text()
    else:
        title = np.nan

    # Getting article url
    if soup.find("meta", {"property": "og:url"}) is not None:
        art_url = soup.find("meta", {"property": "og:url"})["content"]
    elif soup.find("link", rel="canonical"):
        art_url = soup.find("link", rel="canonical")["href"]
    else:
        art_url = url

    # Getting source url
    src_url = BigScraper.get_base_url(art_url)

    # Getting source name
    if soup.find("meta", {"property": "og:site_name"}) is not None:
        src_name = soup.find("meta", {"property": "og:site_name"})["content"]
    else:
        src_name = art_url.split(r"//")
        if "http" in src_name[0]:
            src_name = src_name[1]
        else:
            src_name = src_name[0]
        src_name = src_name.split(r"/")[0]
        for i in ["fr.", "www.", "www2.", ".org", ".fr", ".eu", ".net", ".com"]:
            src_name = src_name.replace(i, "")

    # Source type
    src_type = "xpath_source"

    # Getting image
    if soup.find("meta", {"property": "og:image"}) is not None:
        art_img = soup.find("meta", {"property": "og:image"})["content"]
    else:
        art_img = np.nan

    # Getting author
    if soup.find("meta", {"name": "author"}) is not None:
        art_auth = soup.find("meta", {"name": "author"})["content"].split(",")
    elif soup.find("meta", {"name": "twitter:data1"}) is not None:
        art_auth = soup.find("meta", {'name': "twitter:data1"})[
            "content"].split(",")
    elif soup.find("meta", {"property": "sage:author"}) is not None:
        art_auth = soup.find("meta", {"property": "sage:author"})[
            "content"].split(",")
    else:
        art_auth = np.nan

    # Getting tags
    if soup.find("meta", {"name": "keywords"}) is not None:
        art_tag = soup.find("meta", {"name": "keywords"})["content"].split(",")
    elif soup.find("meta", {"sage": "sageTags"}) is not None:
        art_tag = soup.find("meta", {"sage": "sageTags"})["content"].split(",")
    elif soup.find("meta", {"property": "article:tag"}) is not None:
        art_tag = soup.find("meta", {"property": "article:tag"})[
            "content"].split(",")
    else:
        art_tag = np.nan

    return [content, content_html_str, date, art_lang, title, art_url, src_url, src_name, src_type, art_img, art_auth, art_tag]

### Différentes façon de récupérer le contenu des articles

***1er cas: Contenu dans la balise article***<br>
La fonction *get_content_article* va chercher toutes les balises qui peuvent contenir du texte dans la balise "article" même si ce contenu est dans différentes balises "div".

In [3]:
def get_content_article(soup):
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li']
    try:
        content_html = soup.find("article")
        try:
            content = "\n".join(tag.text for tag in soup.find("article").find_all(list_balises))
        except:
            content = np.nan
    except:
        content_html = np.nan
        content = np.nan
    return content, content_html

***2ème cas: Contenu dans des balises div qui sont dans une balise article***<br>
La fonction *get_content_div_in_article* récupère toutes les balises qui peuvent contenir du texte dans une balise "div" située dans la balise "article". Si il y a plusieurs balises "div" dans la balise "article", la fonction sélectionne celle qui contient le plus de balises qui contiennent du texte.

In [4]:
def get_content_div_in_article(soup):
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li']
    for tag in soup.find("article").find_all("div"):
        if type(tag) == bs4.element.Tag:
            children_names = [element.name for element in tag.contents if element.name is not None]
            content_html = list()
            if len(set(children_names) & set(list_balises)) != 0:
                content_html.append(tag)
    for div in content_html:
        content = "\n".join(tag.text for tag in div.children if tag.name in list_balises)
    content = content.strip().replace("\xa0", "").replace("\t", "")
    return content, content_html

***3ème cas: Contenu dans une seule balise div parmis d'autres***<br>
La fonction *get_content_max_div* cherche la balise "div" qui contient le contenu de l'article. Pour cela, elle parcourt toutes les balises "div" et vérifie s'il y a une balise qui contient du texte dans ses enfants directs, si c'est le cas, la balise "div" et tout son contenu sont ajoutés à une liste. Enfin pour sélectionner une seule balise "div", comme précédemment, la fonction sélectionne celle qui contient le plus de balises faites pour le texte.

In [5]:
def get_content_max_div(soup):
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li', 'span']
    list_div = list()
    for div in soup.find_all("div"):
        if div.find_all(list_balises, recursive = False):
            list_div.append(div)
    idx_max = np.argmax([len(div.find_all(list_balises)) for div in list_div])
    content_html = list_div[idx_max]
    content = "\n".join(tag.text for tag in content_html.children if tag.name in list_balises).strip().replace("\xa0", "")
    return content, content_html

***4ème cas: Contenu dans plusieurs balises div (on prend la balise qui comprend le plus de balises div contenant du texte)***<br>
La fonction *get_content_all_div* cherche toutes les balises "div" qui ont une balise qui contient du texte parmis les balises enfants. Ensuite, pour sélectionner une seule balise "div", la fonction construit une liste avec toutes les balises parents des balises "div" sélectionnées puis sélectionne la balise parent la plus présente dans la liste.

In [6]:
def get_content_all_div(soup):
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li', 'span']
    list_div = [div for div in soup.find_all("div") if div.find_all(list_balises, recursive = False)] # all div with one element of the list_balises as descendant
    list_parents = [div.parent for div in list_div] # parent tag for each div
    content_html = max(set(list_parents), key = list_parents.count) # most parent tag in list_parents
    content = "\n".join(tag.text for tag in div if tag in list_balises for div in content_html.children if div in list_div)
    return content, content_html

***5ème cas: Contenu recherché à partir des balises contenues***<br>
La fonction *get_content_from_tag* recherche toutes les balises dans le code source qui contiennent le texte. Puis, pour chacune de ces balises, cherche sa balise parent et sélectionne la balise parent la plus présente.

In [7]:
def get_content_from_tag(soup):
    list_balises = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'strong', 'i', 'em',
                    'pre', 'mark', 'small', 'del', 's', 'ins', 'u', 'sub', 'sup', 'dfn', 'p', 'span', 'ul', 'li', 'span']
    list_parents = [tag.parent for tag in soup.find_all(list_balises)]
    content_html = max(set(list_parents), key = list_parents.count)
    content = "\n".join(tag.text for tag in content_html.contents if tag.name in list_balises)
    return content, content_html       

### Test des différentes fonctions de scraping générique pour le contenu

In [8]:
Url = ["https://www.fnccr.asso.fr/article/big-data-territorial-publication-de-letude-de-la-fnccr/",
      "https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45",
      "https://www.lemondeinformatique.fr/actualites/lire-les-salaries-et-les-dirigeants-percoivent-differement-la-transformation-digitale-81352.html",
      "https://www.cnil.fr/fr/les-collectivites-territoriales-et-lopen-data-concilier-ouverture-des-donnees-et-protection-des",
      "https://www.inserm.fr/actualites-et-evenements/actualites/ondes-electromagnetiques-faut-il-craindre-5g",
      "https://www.parlonsrh.com/raisons-utiliser-lintelligence-artificielle-dans-gestion-gpec/",
      "https://www.myrhline.com/actualite-rh/de-la-gpec-et-au-workforce-planning-les-5-evolutions-a-connaitre.html",
      "https://grh-multi.net/fr/2016/05/compte-rendu-de-levenement-big-data-gpec/",
      "https://changethework.com/gestion-paie-innovations/",
      "https://changethework.com/chatbot-rh-recrutement/",
      "http://sabbar.fr/management/le-management-strategique-et-le-management-operationnel/#:~:text=Le%20management%20op%C3%A9rationnel%20correspond%20aux,pour%20atteindre%20les%20objectifs%20fix%C3%A9s.",
      "https://blockchainfrance.net/decouvrir-la-blockchain/c-est-quoi-la-blockchain/",
      "https://citoyen-ne-s-de-marseille.fr/cest-quoi-les-autorisations-de-programmes/",
      "https://hellofuture.orange.com/fr/mot-de-linnovation-brain-computer-interface/",
      "https://omicron-formation.fr/3-gestion-et-organisation-de-la-cybersecurite/",
      "https://www.data.gouv.fr/fr/datasets/les-elus-municipaux/",
      "https://www.data.gouv.fr/fr/datasets/repertoire-national-des-elus-1/",
      "https://www.digitalrecruiters.com/blog/ia-et-machine-learning-comment-optimiser-en-profondeur-les-processus-rh.html",
      "https://www.erudit.org/fr/revues/ipme/2014-v27-n2-ipme01485/1026072ar/",
      "https://www.lebigdata.fr/base-de-donnees",
      "https://www.linternaute.fr/dictionnaire/fr/definition/agglomeration/",
      "https://www.linternaute.fr/dictionnaire/fr/definition/software/",
      "https://www.riskinsight-wavestone.com/2019/08/detecter-incidents-machine-learning/",
      "https://www.sap.com/france/products/erp-financial-management/grc.html",
      "https://www.cadre-dirigeant-magazine.com/manager/la-recherche-operationnelle-un-formidable-outil-daide-a-la-decision/"]

In [10]:
df_content = pd.DataFrame(columns = ["article", "div_in_article", "one_div", "all_div", "from_tag"])
df_content_html = pd.DataFrame(columns = ["article", "div_in_article", "one_div", "all_div", "from_tag"])
for url in Url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    list_func = [get_content_article, get_content_div_in_article, get_content_max_div, get_content_all_div, get_content_from_tag]
    list_content = list()
    list_content_html = list()
    for func in list_func:
        try:
            content, content_html = func(soup)
        except:
            content, content_html = np.nan, np.nan
        list_content.append(content)
        list_content_html.append(content_html)
    df_content.loc[len(df_content)] = list_content
    df_content_html.loc[len(df_content_html)] = list_content_html

In [12]:
df_content

,article,div_in_article,one_div,all_div,from_tag
0,Si les regards se tournent souvent vers les gr...,NaN,Nos compétences\n\nÉnergie\n\nPrésentation\nRé...,NaN,Si les regards se tournent souvent vers les gr...
1,Comment tuer l’innovation avec l’analyse finan...,Fiche de lecture\nInnovation killers : how fin...,Fiche de lecture\nInnovation killers : how fin...,NaN,Fiche de lecture\nInnovation killers : how fin...
2,/ / Transformation numérique Les salariés et l...,NaN,Une étude réalisée par l'ESN Inetum avec l'Ins...,NaN,/ MOBILITÉ\n/ OS\n/ PME\n/ RÉSEAU\n/ SÉCURITÉ\...
3,NaN,NaN,Quelles sont les obligations de publication ?\...,NaN,Quelles sont les obligations de publication ?\...
4,Ondes électromagnétiques : Faut-il craindre la...,NaN,Connaître l'Inserm\n\n\n\n\n\n\nConnaître l'...,NaN,\nLa recherche à l'Inserm\n\n\nLe continuum d...
5,\n\n\n\t\t\t\t\t\t\tLe 26\nMai\n2020‚ \n\t\t\t...,NaN,Expertises\n\nConseil en Marketing et Communic...,NaN,La bonne connaissance des besoins de l’entrepr...
6,NaN,NaN,"Historiquement, la GPEC est un exercice mené d...",NaN,"Historiquement, la GPEC est un exercice mené d..."
7,Compte Rendu de l’évènement : Big Data & GPEC ...,NaN,,NaN,Compte Rendu de l’évènement : Big Data & GPEC\...
8,\n\n\t\t\t\tMieux vivre la ménopause au travai...,"Aurélien Leleux\n\nJanuary 20, 2020",L’innovation en RH est un thème de plus en plu...,NaN,L’innovation en RH est un thème de plus en plu...
9,\n\n\t\t\t\tMieux vivre la ménopause au travai...,"Aurélien Leleux\n\nJanuary 20, 2020","Les chatbots, ou agents relationnels, sont dep...",NaN,"Les chatbots, ou agents relationnels, sont dep..."


La dernière fonction est celle qui renvoie le plus de résultats mais certains ne correspondent pas au contenu de l'article. Elle sera tout de même utilisée dans le fichier principal pour récupérer le contenu.